In [12]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
import plotly.graph_objects as go

# Function to fetch data
def fetch_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data[['Close']]

# Function to preprocess data
def preprocess_data(data, lookback=60):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    return X, y, scaler

# Function to build the enhanced LSTM-GRU model
def build_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.3),
        GRU(64, return_sequences=True),
        Dropout(0.3),
        LSTM(32, return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Function to plot predictions with an improved UI
def plot_predictions_with_ui(actual, predicted, dates):
    fig = go.Figure()

    # Add real data
    fig.add_trace(go.Scatter(
        x=dates,
        y=actual.flatten(),
        mode='lines',
        name='Actual Prices',
        line=dict(color='green', width=2)
    ))

    # Add predicted data
    fig.add_trace(go.Scatter(
        x=dates,
        y=predicted.flatten(),
        mode='lines',
        name='Predicted Prices',
        line=dict(color='magenta', width=2)
    ))

    # Update layout for dark theme and hover info
    fig.update_layout(
        title='Stock Price Prediction',
        xaxis_title='Date',
        yaxis_title='Price',
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        hovermode='x unified'
    )

    fig.show()

# Main function
def main():
    # Parameters
    ticker = '^NSEI'  # NIFTY 50 Index ticker symbol
    start_date = '2018-01-01'  # Data from the last 5 years
    end_date = '2023-01-01'
    lookback = 60

    # Step 1: Fetch data
    data = fetch_data(ticker, start_date, end_date)

    # Step 2: Preprocess data
    X, y, scaler = preprocess_data(data.values, lookback)

    # Step 3: Train-test split
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Step 4: Build and train the model
    model = build_model((X_train.shape[1], 1))
    model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1)

    # Step 5: Make predictions
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Extract corresponding dates for test data
    test_dates = data.index[-len(y_test):]

    # Step 6: Plot predictions with improved UI
    plot_predictions_with_ui(actual_prices, predictions, test_dates)

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed

Epoch 1/50



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



53/53 ━━━━━━━━━━━━━━━━━━━━ 12s 93ms/step - loss: 0.0571 - val_loss: 0.0038
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 97ms/step - loss: 0.0046 - val_loss: 0.0039
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 9s 81ms/step - loss: 0.0040 - val_loss: 0.0024
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/step - loss: 0.0038 - val_loss: 0.0013
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 9s 81ms/step - loss: 0.0034 - val_loss: 0.0011
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 6s 108ms/step - loss: 0.0026 - val_loss: 0.0018
Epoch 7/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 9s 91ms/step - loss: 0.0024 - val_loss: 0.0010
Epoch 8/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - loss: 0.0022 - val_loss: 9.4934e-04
Epoch 9/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step - loss: 0.0027 - val_loss: 0.0029
Epoch 10/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 9s 80ms/step - loss: 0.0022 - val_loss: 0.0024
Epoch 11/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 108ms/step - loss: 0.0017 - val_loss: 0.0029
Epoch 12/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - loss: 0.00